<a href="https://colab.research.google.com/github/S1R3S1D/Image_Classification_DL/blob/main/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torch import nn, optim
from tqdm import tqdm

In [23]:
from torch._C import dtype
train_data = datasets.CIFAR10('./data', True, download=True, transform=transforms.Compose([transforms.ToTensor(),
                                                                                          transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                                                                               (0.2470, 0.2435, 0.2616))]),
                              target_transform=lambda y: torch.zeros(10, dtype=torch.float).scatter(0, torch.tensor(y), value=1))
test_data = datasets.CIFAR10('./data', False, download=True, transform=transforms.Compose([transforms.ToTensor()]),)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified


In [33]:
class ConvNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1    = nn.Conv2d(3, 16, 3, padding=1)
    self.ac1      = nn.Tanh()
    self.pool1    = nn.MaxPool2d(2)

    self.conv2    = nn.Conv2d(16, 8, 3, padding=1)
    self.ac2      = nn.Tanh()
    self.pool2    = nn.MaxPool2d(2)

    self.fc1      = nn.Linear(8*8*8, 1024)
    self.ac3      = nn.Tanh()

    self.fc2      = nn.Linear(1024, 10)
    self.ac4      = nn.Softmax()
  
  def forward(self, x):
    out   = self.pool1(self.ac1(self.conv1(x)))
    out   = self.pool2(self.ac2(self.conv2(out)))
    out   = out.view(-1, 8*8*8)
    out   = self.ac3(self.fc1(out))
    out   = self.fc2(out)
    return out

In [34]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print('Training on :', device)

Training on : cuda


In [35]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
  for epoch in tqdm(range(n_epochs)):
    losses = 0.0
    for imgs, labels in train_loader:
      imgs = imgs.to(device)
      labels = labels.to(device)

      outputs = model(imgs)

      loss = loss_fn(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      losses+=loss.item()

    if(epoch==1 or (epoch+1)%10==0 ):
      print('Epoch ', epoch, ' Loss : ', losses)

In [36]:
n_epochs = 100
model = ConvNet().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

training_loop(n_epochs, optimizer, model, loss_fn, train_loader)

  2%|▏         | 2/100 [00:25<20:57, 12.83s/it]

Epoch  1  Loss :  1262.9515316486359


 10%|█         | 10/100 [02:08<19:15, 12.83s/it]

Epoch  9  Loss :  898.2962037324905


 20%|██        | 20/100 [04:16<17:05, 12.82s/it]

...

Epoch  79  Loss :  405.3643242716789


 90%|█████████ | 90/100 [19:17<02:08, 12.85s/it]

Epoch  89  Loss :  331.2527230232954


100%|██████████| 100/100 [21:26<00:00, 12.87s/it]

Epoch  99  Loss :  260.12037339806557


In [42]:
#Validation

def validation_loop(test_loader, model):
  done = 0
  with torch.no_grad():
    tot = 0
    for imgs, labels in test_loader:
      imgs = imgs.to(device)
      outputs = model(imgs)
      
      
      for i in range(outputs.shape[0]):
        tot=tot+1
        if(torch.argmax(outputs[i]).item()==labels[i]):
          done=done+1
  accuracy = done/tot * 100
  print('Accuracy :', accuracy, '% over', tot, 'images')


In [43]:
validation_loop(test_loader, model)

Accuracy : 24.43 % over 10000 images
